# Databricks Demo: Iceberg Tables
---

## What is Apache Iceberg?
"Apache Iceberg is an open-source, high-performance table format for large analytical datasets in data lakes, providing database-like reliability, consistency, and features like schema evolution and time travel. It acts as a metadata layer on top of data files (like Parquet), enabling ACID transactions and simplifying data management for data lakehouse architectures by giving data lakes SQL table-like capabilities".

It's essentially an open-source table format that is efficient and compatible between different systems and platforms.

## Setting up Databricks
---


### Step 1: Create a Catalog
* Databricks's fancy word for a "database"
* We did this step in the Databricks UI

In [0]:
CREATE CATALOG IF NOT EXISTS uprm_big_data
MANAGED LOCATION '<S3-url>';  -- Specify the path to the S3 bucket.

In [0]:
USE CATALOG uprm_big_data;

### Step 2: Create a Schema
* A location to hold our tables

In [0]:
CREATE SCHEMA IF NOT EXISTS bad_bunny;
USE SCHEMA bad_bunny;

## Creating an Iceberg table
---

### Step 1: Getting data
* We used a tool like the [Youtube Scraper Agent](https://console.apify.com/actors/h7sDV53CddomktSi5/information/latest/readme) from [Apify](https://apify.com/) to get metadata from videos.
  * For this example, we are using Bad Bunny music videos.
* Then, we get the generated JSON outputs from the scraper and put them in an [Amazon S3](https://aws.amazon.com/s3/$0) bucket.

### Step 2: Reading JSONs with Databricks

In [0]:
-- Creates a temp view of the JSON files
CREATE OR REPLACE TEMP VIEW v_mv_metadata
USING json
OPTIONS (
  path 's3://uprm-2025-demo-yelp/bad_bunny_mv_metadata/'
  , multiLine 'true'                -- Important if JSON spans multiple lines or arrays
  , escapeInvalidCharacters 'true'
);

### Step 3: Turning the JSON into an Iceberg table

In [0]:
CREATE OR REPLACE TABLE mv_metadata
USING ICEBERG  -- Table format (Delta by default)
TBLPROPERTIES ('format-version' = '2')  -- enable Iceberg v2 features
AS
SELECT * FROM v_mv_metadata;

**Previewing the table:**

In [0]:
SELECT * FROM bad_bunny.mv_metadata;

## Features of Iceberg tables
---

Apache Iceberg is like a “universal data table format” that allows many different data tools and clouds to share the same data safely and efficiently.

### - Exploring table properties
Here we can see attributes related to the table, like:
* Column names, thier data types, and any commnets attached to them
* Detailed table information (metadata)

In [0]:
DESCRIBE bad_bunny.mv_metadata;  -- See attributes and data types

In [0]:
DESCRIBE EXTENDED bad_bunny.mv_metadata;  -- See columns and metadata

### - Time Travel
One of the most useful features Iceberg provides is _time traveling_! i.e., **version history**

Just like a version control system tool like Git, you can keep track of the different versions of a table based on each time it is changed. This includes:
* Replacing the table
* Create, update, delete operations
* Insertions or merges
* Truncations
* Among other mutating SQL operations

#### Looking at the table history

In [0]:
DESCRIBE HISTORY bad_bunny.mv_metadata;

#### Trying out the versions!

In [0]:
-- Preview Schema
SELECT * FROM bad_bunny.mv_metadata LIMIT 5;

* Since we know all the videos here are from Bad Bunny, we don't really care about the `channelId`, `channelName`, or `channelUsername` columns. So, we could drop them.

In [0]:
-- Drop columns
ALTER TABLE bad_bunny.mv_metadata DROP COLUMNS (channelId, channelName, channelUrl, channelUsername);

-- Preview schema
SELECT * FROM bad_bunny.mv_metadata LIMIT 5;

* We can see the table no longer has the columns. **But what if we wanted to restore them?**
* Here is where we can use time travel!

In [0]:
-- See the version history
DESCRIBE HISTORY bad_bunny.mv_metadata;

* Above, we can see the different verions, their date, and what operation generated that version.
* Below, we can see the previous version of the table.

In [0]:
-- Query the table version you need
SELECT * FROM bad_bunny.mv_metadata VERSION AS OF 3 LIMIT 5;

-- You could also query versions by timestamp:
-- SELECT * FROM bad_bunny.mv_metadata TIMESTAMP AS OF '2025-10-08' LIMIT 5;